In [1]:
import pandas as pd
import numpy as np

# Multi level index in pandas
- How to create a Series with a MultiIndex, and how to convert it to a DataFrame
- How to select from a Series with a MultiIndex
- How to create a DataFrame with a MultiIndex
- How to select from a DataFrame with a MultiIndex
- How to merge two DataFrames with MultiIndexes

In [2]:
stocks = pd.read_csv('stocks.csv')
stocks

,Date,Close,Volume,Symbol
0,2016-10-03,31.50,14070500,CSCO
1,2016-10-03,112.52,21701800,AAPL
2,2016-10-03,57.42,19189500,MSFT
3,2016-10-04,113.00,29736800,AAPL
4,2016-10-04,57.24,20085900,MSFT
5,2016-10-04,31.35,18460400,CSCO
6,2016-10-05,57.64,16726400,MSFT
7,2016-10-05,31.59,11808600,CSCO
8,2016-10-05,113.05,21453100,AAPL


In [3]:
stocks.index

RangeIndex(start=0, stop=9, step=1)

## How to create a Series with a MultiIndex, and how to convert it to a DataFrame

Q) For each symbol(company) what is the mean closing price?

In [4]:
stocks.groupby('Symbol').Close.mean()

Symbol
AAPL    112.856667
CSCO     31.480000
MSFT     57.433333
Name: Close, dtype: float64

Q) For each pair of symbol(company) and date. What is the mean closing price?

In [5]:
ser = stocks.groupby(['Symbol', 'Date']).Close.mean()
ser

Symbol  Date      
AAPL    2016-10-03    112.52
        2016-10-04    113.00
        2016-10-05    113.05
CSCO    2016-10-03     31.50
        2016-10-04     31.35
        2016-10-05     31.59
MSFT    2016-10-03     57.42
        2016-10-04     57.24
        2016-10-05     57.64
Name: Close, dtype: float64

Here, ser is a pandas series with multi-index.

In [6]:
ser.index

MultiIndex(levels=[['AAPL', 'CSCO', 'MSFT'], ['2016-10-03', '2016-10-04', '2016-10-05']],
           codes=[[0, 0, 0, 1, 1, 1, 2, 2, 2], [0, 1, 2, 0, 1, 2, 0, 1, 2]],
           names=['Symbol', 'Date'])

In [7]:
type(ser)

pandas.core.series.Series

multi-index adds one more dimension to the series.  

You can unstack a multi-index series and it can become a dataframe. As series with mult-index is two-dimension so it can naturally be represented as dataframe.

In [8]:
ser.unstack()

Date,2016-10-03,2016-10-04,2016-10-05
Symbol,,,
AAPL,112.52,113.00,113.05
CSCO,31.50,31.35,31.59
MSFT,57.42,57.24,57.64


The same dataframe can be obtained by using __pivot_table()__. i.e., without using groupby and unstack.  
Default aggreggate function for __pivot_table()__ is __mean__.

In [9]:
df = stocks.pivot_table(values = 'Close', index = 'Symbol', columns = 'Date')
df

Date,2016-10-03,2016-10-04,2016-10-05
Symbol,,,
AAPL,112.52,113.00,113.05
CSCO,31.50,31.35,31.59
MSFT,57.42,57.24,57.64


## How to select from a Series with a MultiIndex

In [10]:
ser

Symbol  Date      
AAPL    2016-10-03    112.52
        2016-10-04    113.00
        2016-10-05    113.05
CSCO    2016-10-03     31.50
        2016-10-04     31.35
        2016-10-05     31.59
MSFT    2016-10-03     57.42
        2016-10-04     57.24
        2016-10-05     57.64
Name: Close, dtype: float64

To select only AAPL data with Date and closing price.

In [11]:
ser.loc['AAPL']  # 1st/outer level index value-AAPL

Date
2016-10-03    112.52
2016-10-04    113.00
2016-10-05    113.05
Name: Close, dtype: float64

To select only AAPL data with Date = '2016-10-03' and closing price.

In [12]:
ser.loc['AAPL', '2016-10-03']

112.52

To select only Date = '2016-10-03' : Symbol and closing price. Here Date is 2nd/inner level index. Observe usage of loc.

In [13]:
ser.loc[:, '2016-10-03']

Symbol
AAPL    112.52
CSCO     31.50
MSFT     57.42
Name: Close, dtype: float64

If u have reshaped the dataframe to look as dataframe as below. The above commands of loc will work the same as above for the hence created dataframe.

In [15]:
df

Date,2016-10-03,2016-10-04,2016-10-05
Symbol,,,
AAPL,112.52,113.00,113.05
CSCO,31.50,31.35,31.59
MSFT,57.42,57.24,57.64


## How to create a DataFrame with a MultiIndex

In [16]:
# using the stocks dataset
stocks.set_index(['Symbol', 'Date'], inplace = True)
stocks

,,Close,Volume
Symbol,Date,,
CSCO,2016-10-03,31.50,14070500
AAPL,2016-10-03,112.52,21701800
MSFT,2016-10-03,57.42,19189500
AAPL,2016-10-04,113.00,29736800
MSFT,2016-10-04,57.24,20085900
CSCO,2016-10-04,31.35,18460400
MSFT,2016-10-05,57.64,16726400
CSCO,2016-10-05,31.59,11808600
AAPL,2016-10-05,113.05,21453100


In [17]:
stocks.index

MultiIndex(levels=[['AAPL', 'CSCO', 'MSFT'], ['2016-10-03', '2016-10-04', '2016-10-05']],
           codes=[[1, 0, 2, 0, 2, 1, 2, 1, 0], [0, 0, 0, 1, 1, 1, 2, 2, 2]],
           names=['Symbol', 'Date'])

We can also sort the index - sorting is done first on the outer index level and then on the inner index level.

In [18]:
stocks.sort_index(inplace = True)
stocks

Close    Volume
Symbol Date                        
AAPL   2016-10-03  112.52  21701800
       2016-10-04  113.00  29736800
       2016-10-05  113.05  21453100
CSCO   2016-10-03   31.50  14070500
       2016-10-04   31.35  18460400
       2016-10-05   31.59  11808600
MSFT   2016-10-03   57.42  19189500
       2016-10-04   57.24  20085900
       2016-10-05   57.64  16726400

In [19]:
# to see only for AAPL
stocks.loc['AAPL']

,Close,Volume
Date,,
2016-10-03,112.52,21701800
2016-10-04,113.00,29736800
2016-10-05,113.05,21453100


## How to select from a DataFrame with a MultiIndex

In [20]:
# to obtain AAPL and oct 3rd data. Here confusion might arise whether one id for row and other for columns
stocks.loc['AAPL', '2016-10-03']

Close          112.52
Volume    21701800.00
Name: (AAPL, 2016-10-03), dtype: float64

In [21]:
## so better to use this format 
stocks.loc[('AAPL', '2016-10-03'),:]

Close          112.52
Volume    21701800.00
Name: (AAPL, 2016-10-03), dtype: float64

In [22]:
# Now to get value for CLose columns
stocks.loc[('AAPL', '2016-10-03'),'Close']

112.52

In [23]:
# Now to get data for AAPL and MSFT and 3rd oct
stocks.loc[(['AAPL', 'MSFT'], '2016-10-03'),:]

,,Close,Volume
Symbol,Date,,
AAPL,2016-10-03,112.52,21701800
MSFT,2016-10-03,57.42,19189500


Now to get 2 dates 3rd and 4th october for all symbols. here `stocks.loc[(:, ['2016-10-03', '2016-10-04']),:]` wont work (technical issue) so need to use `slice(None)` inplace of __:__

In [24]:
stocks.loc[(slice(None), ['2016-10-03', '2016-10-04']),:]

Close    Volume
Symbol Date                        
AAPL   2016-10-03  112.52  21701800
       2016-10-04  113.00  29736800
CSCO   2016-10-03   31.50  14070500
       2016-10-04   31.35  18460400
MSFT   2016-10-03   57.42  19189500
       2016-10-04   57.24  20085900

## How to merge two DataFrames with MultiIndexes

In [32]:
close = pd.read_csv('stocks.csv', usecols = [0,1,3], index_col = ['Symbol', 'Date']).sort_index()
close

Close
Symbol Date              
AAPL   2016-10-03  112.52
       2016-10-04  113.00
       2016-10-05  113.05
CSCO   2016-10-03   31.50
       2016-10-04   31.35
       2016-10-05   31.59
MSFT   2016-10-03   57.42
       2016-10-04   57.24
       2016-10-05   57.64

In [29]:
volume = pd.read_csv('stocks.csv', usecols = [0,2,3], index_col = ['Symbol', 'Date']).sort_index()
volume

Volume
Symbol Date                
AAPL   2016-10-03  21701800
       2016-10-04  29736800
       2016-10-05  21453100
CSCO   2016-10-03  14070500
       2016-10-04  18460400
       2016-10-05  11808600
MSFT   2016-10-03  19189500
       2016-10-04  20085900
       2016-10-05  16726400

To merge Close and Volume dataframe.  
Here we will be merging based on the indices of left and right dataframe using parameters to the close and volume dataframe.
__left_index__ and __right_index__.

In [33]:
both = pd.merge(close, volume, left_index = True, right_index = True)
both

Close    Volume
Symbol Date                        
AAPL   2016-10-03  112.52  21701800
       2016-10-04  113.00  29736800
       2016-10-05  113.05  21453100
CSCO   2016-10-03   31.50  14070500
       2016-10-04   31.35  18460400
       2016-10-05   31.59  11808600
MSFT   2016-10-03   57.42  19189500
       2016-10-04   57.24  20085900
       2016-10-05   57.64  16726400

In [34]:
#  Can go back to tidy data for using reset_index
both.reset_index()

,Symbol,Date,Close,Volume
0,AAPL,2016-10-03,112.52,21701800
1,AAPL,2016-10-04,113.00,29736800
2,AAPL,2016-10-05,113.05,21453100
3,CSCO,2016-10-03,31.50,14070500
4,CSCO,2016-10-04,31.35,18460400
5,CSCO,2016-10-05,31.59,11808600
6,MSFT,2016-10-03,57.42,19189500
7,MSFT,2016-10-04,57.24,20085900
8,MSFT,2016-10-05,57.64,16726400
